In [1]:
from ML_Preparation.Preprocessing_M import DataExtractor, DataPreprocessing

de = DataExtractor("D:/hka-aqm-m", "C:/Users/danie/building_M_data")

df = de.create_df()

No .dat files found in C:/Users/danie/building_M_data. 
 Trying to extract files from the original directory D:/hka-aqm-m
Read data successfully.
Data contains 979166 data points and 18 columns.


In [2]:
dp = DataPreprocessing()

df_preprocessed = dp.preprocess_df(df, rolling_window = "7d", sample_time = "1d")

Number of outliers detected: -73438
Number of normal samples detected: 905728


In [3]:
df_preprocessed.columns

Index(['date_time', 'tmp', 'hum', 'CO2', 'VOC', 'vis', 'IR', 'BLE', 'tavg',
       'tmin', 'tmax', 'wdir', 'wspd', 'wpgt', 'pres', 'room_number',
       'tmp_diff', 'hum_diff', 'VOC_diff', 'vis_diff', 'IR_diff', 'BLE_diff',
       'tavg_diff', 'tmin_diff', 'tmax_diff', 'wdir_diff', 'wspd_diff',
       'wpgt_diff', 'pres_diff', 'year', 'dayofweek', 'hour', 'season',
       'VOC_CO2_ratio'],
      dtype='object')

In [4]:
# df_preprocessed.to_parquet("hka_M_preprocessed.parquet")

In [5]:
# import pandas as pd
# df_preprocessed = pd.read_parquet("hka_M_preprocessed.parquet")

In [6]:
# from Feature_Engineering import *
# fe = FeatureEngineering(df_preprocessed.drop(columns = ["VOC_diff", "VOC_diff_per_sec", "VOC_CO2_ratio"]), label = "CO2", categorical_features = ["season", "room_number", "dayofweek", "color"])

In [7]:
from ML_Preparation.Feature_Engineering import *
fe = FeatureEngineering(df_preprocessed, label = "CO2", categorical_features = ["season", "room_number", "dayofweek", "color"])

'color'


In [8]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(fe.X_train, fe.y_train)

LinearRegression()

In [9]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

pred_test = lr.predict(fe.X_test)

pred_train = lr.predict(fe.X_train)

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_train, y_pred = pred_train)}")

print(f"Training score: {mean_absolute_percentage_error(y_true = fe.y_test, y_pred = pred_test)}")

Training score: 0.06554519823717572
Training score: 2400066858.9374


In [10]:
fe.X_train.shape

(5670, 67)

In [11]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Dropout, Bidirectional
from keras.optimizers import Adam, SGD

class MyLSTMModel:
    def __init__(self, lstm_units=int(fe.X_train.shape[1]),input_shape=(None, fe.X_train.shape[1])):
        self.model = Sequential()
        self.model.add(LSTM(units=lstm_units, input_shape=input_shape, return_sequences = True, recurrent_dropout = 0.3))
        self.model.add(Dropout(0.3))
        self.model.add(Bidirectional(LSTM(units=lstm_units, recurrent_dropout = 0.3)))
        self.model.add(Dense(60, activation = "relu"))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(40, activation = "relu"))
        self.model.add(Dropout(0.3))
        self.model.add(Dense(20, activation = "relu"))
        self.model.add(Dropout(0.3))

        self.model.add(Dense(1))

        self.model.compile(optimizer=Adam(learning_rate = 0.05), loss='mse')
    
    def summary(self):
        self.model.summary()

# Instantiate the model
my_model = MyLSTMModel()

# Print the model summary
X_train_array = fe.X_train.values.reshape(fe.X_train.shape[0], 1, fe.X_train.shape[1]).astype(np.float32)
y_train_array = fe.y_train.values.astype(np.float32).reshape(fe.y_train.shape[0], 1, 1)
my_model.model.fit(X_train_array, y_train_array, epochs = 15, batch_size = 200)

Epoch 1/15


c:\Users\danie\anaconda3\envs\awp1\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 164755.7031
Epoch 2/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30765.5781
Epoch 3/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34328.1211
Epoch 4/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28757.6816 
Epoch 5/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27794.8633 
Epoch 6/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27792.9746 
Epoch 7/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27343.4336 
Epoch 8/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24804.1172 
Epoch 9/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25228.8125 
Epoch 10/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28674.9062 
Epoch 11/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 27973.7363 
Epoch 12/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25329.9121 
Epoch 13/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26072.9805 
Epoch 14/15
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24048.7090 
Epoch 15/15
29/29 ━━━━━━━━━━

In [12]:
X_test_array = fe.X_test.values.reshape(fe.X_test.shape[0], 1, fe.X_test.shape[1]).astype(np.float32)
y_test_array = fe.y_test.values.astype(np.float32).reshape(fe.y_test.shape[0], 1, 1)
pred = my_model.model.predict(X_test_array)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [13]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error

def evaluate_model(y_true, y_pred):
    eval_df = dict()

    eval_df["r2"] = r2_score(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mape"] = mean_absolute_percentage_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mae"] = mean_absolute_error(y_pred = y_pred, y_true = y_true).round(2)
    eval_df["mse"] = mean_squared_error(y_pred = y_pred, y_true = y_true).round(2)

    return pd.DataFrame([eval_df])

evaluate_model(fe.y_test, pred)


,r2,mape,mae,mse
0,-0.28,0.14,65.46,5294.79


In [14]:
evaluate_model(fe.y_train, my_model.model.predict(X_train_array))

178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step


,r2,mape,mae,mse
0,0.73,0.12,73.85,7484.02


In [15]:
test_df = fe.df.loc[fe.y_test.index, :]

In [16]:
import plotly.express as px

px.scatter(x = fe.y_test.index, y = [fe.y_test, pd.DataFrame(pred)[0]])

In [17]:
# X_test_array = fe.X_test.values.reshape(fe.X_test.shape[0], 1, fe.X_test.shape[1]).astype(np.float32)
# y_test_array = fe.y_test.values.astype(np.float32).reshape(fe.y_test.shape[0], 1, 1)
# pred = my_model.model.predict(X_test_array)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step


In [18]:
# fe.df = df_preprocessed.drop("CO2", axis = 1)
# x_deploy = fe.feature_engineering(x_for_prediction = fe.df, only_predict = True)

'color'


In [26]:
import pickle

with open('feature_engineerer.pickle', 'wb') as handle:
    pickle.dump(fe, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
with open('feature_engineerer.pickle', 'rb') as handle:
    fe = pickle.load(handle)

In [33]:
from Deployment.ML_Deployment import *

fe.df = df.copy()

deployer = Predictor(data = df, feature_engineering_class_object = fe, label = "CO2", is_forecast = True, roll = True)

'color'


In [34]:
for col in deployer.x.columns:
    if col not in fe.X_train.columns:
        print(col)

CO2


In [35]:
deployer.predict(x = deployer.x.drop(columns = ["CO2"], axis = 1), model = my_model.model)

201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step


,prediction,room_number
date_time,,
2022-06-01,878.660645,m001
2022-06-02,894.447144,m001
2022-06-03,710.176636,m001
2022-06-04,450.368622,m001
2022-06-20,447.755219,m104
...,...,...
2023-09-26,469.785217,m306
2023-09-26,476.615662,m301
2023-09-26,507.304504,m003


In [36]:
deployer.predict(x = deployer.x.head(5).drop(columns = ["CO2"], axis = 1), model = my_model.model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


,prediction,room_number
date_time,,
2022-06-01,878.660645,m001
2022-06-02,894.447144,m001
2022-06-03,710.176636,m001
2022-06-04,450.368622,m001
2022-06-20,447.755219,m104
